In [1]:
from brian2 import *
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context='poster', style='whitegrid')
%matplotlib inline

# Nengo Interface

In [2]:
from nengo.neurons import *
from nengo.builder.neurons import *
from brian2 import *
import nengo
from nengo.dists import Uniform
from nengo.solvers import NoSolver

class WilsonNeuron(NeuronType):
    """
    Wilson Neuron Model.
    
    Description
    -----------
    
    TODO
    
    Paramters
    ---------
    
    time constants in seconds
    voltages in mV
    
    References
    ----------
    [1] Wilson, Hugh R. "Simplified dynamics of human and mammalian neocortical neurons."
        Journal of theoretical biology 200.4 (1999): 375-388.

    
    """
    
    probeable = ('spikes', 'voltage', 'recovery', 'conductance')
    tauv = NumberParam('tauv')
    taur = NumberParam('taur')
    tauc = NumberParam('tauc')
    
    def __init__(self, thr=-0.20*mV, tauv=0.00097, taur=0.0056, tauc=0.0990):
        super(WilsonNeuron, self).__init__()
        self.thr = thr
        self.tauv = tauv
        self.taur = taur
        self.tauc = tauc
        mag = mamp
        res = ohm
        freq = 10*pi*Hz
        antiunit = siemens/farad
        self.namespace = {'tauv': self.tauv, 'taur': self.taur, 'tauc': self.tauc,
            'thr': self.thr, 'mag': mag, 'res': res, 'antiunit': antiunit, 'freq': freq}

        self.eqns = '''
        dv/dt = antiunit/tauv * (-((17.81 + 47.58*(v/mV) + 33.80*(v/mV)**2)*(v - 0.48*mV))
                            - 26*r*(v + 0.95*mV)
                            - 13*c*(v + 0.95*mV) + I*res) : volt
        dr/dt = Hz/taur * (-r + 1.29*(v/mV) + 0.79 + 3.30*((v/mV) + 0.38)**2) : 1
        dc/dt = Hz/tauc * (-c + 11*((v/mV) + 0.754)*((v/mV) + 0.69)) : 1
        I : amp  # = mag*sin(freq*t)
        '''
        
    @property
    def _argreprs(self):
        args = []
        def add(attr, default):
            if getattr(self, attr) != default:
                args.append("%s=%s" %(attr, getattr(self, attr)))
        add("tauv", 0.00097)
        add("taur", 0.0056)
        add("tauh", 0.0990)
        return args
        
    def rates(self, x, gain, bias):
        return x  # todo
    
    def gain_bias(self, max_rates, intercepts):
        return np.ones(len(max_rates)), np.ones(len(max_rates))  # todo
    
    def step_math(self, dt, briannet, brianens, brianmon, I, output, voltage, time):
        brianens.I = I*mamp
        briannet.run(dt*second)
        voltage[:] = brianmon.v[:,-1]/mV

In [3]:
class SimBrianNeurons(Operator):
    def __init__(self, neuron_type, neurons, I, output, states):
        super(SimBrianNeurons, self).__init__()
        self.neuron_type = neuron_type
        self.neurons = neurons        
        self.reads = [states[0], I]
        self.sets = [output, states[1]]
        self.updates = []
        self.incs = []
        
        self.neurons.v = states[1].initial_value*mV
        self.neurons.r = states[2].initial_value
        self.neurons.c = states[3].initial_value
        self.brianens = neurons
        self.brianmon = StateMonitor(self.neurons, 'v', record=True)
        self.briannet = Network(self.brianens, self.brianmon)  # , update_I

    def make_step(self, signals, dt, rng):
        current = signals[self.current]
        output = signals[self.output]
        voltage = signals[self.voltage]
        time = signals[self.time]
        def step_nrn():
            self.neuron_type.step_math(dt, self.briannet, self.brianens, self.brianmon, current, output, voltage, time)
        return step_nrn

    @property
    def time(self):
        return self.reads[0]
    @property
    def current(self):
        return self.reads[1]
    @property
    def output(self):
        return self.sets[0]
    @property
    def voltage(self):
        return self.sets[1]


@Builder.register(WilsonNeuron)
def build_wilsonneuron(model, neuron_type, neurons):
    
    model.sig[neurons]['voltage'] = Signal(
        -0.754*np.ones(neurons.size_in), name="%s.voltage" % neurons)
    model.sig[neurons]['recovery'] = Signal(
        0.279*np.ones(neurons.size_in), name="%s.recovery" % neurons)
    model.sig[neurons]['conductance'] = Signal(
        0.0*np.ones(neurons.size_in), name="%s.conductance" % neurons)
    thr = neuron_type.thr*mV
    
    brianens = NeuronGroup(neurons.size_in, neuron_type.eqns,
        threshold='v > thr', namespace=neuron_type.namespace, method='rk4')
    
    model.add_op(SimBrianNeurons(
        neuron_type=neuron_type,
        neurons=brianens,
        I=model.sig[neurons]['in'],
        output=model.sig[neurons]['out'],
        states=[
            model.time,
            model.sig[neurons]['voltage'],
            model.sig[neurons]['recovery'],
            model.sig[neurons]['conductance']]))

In [ ]:
n_neurons = 10
w = 1e-3 * np.random.uniform(-1, 1, size=(n_neurons, n_neurons))

with nengo.Network() as network:
    stim = nengo.Node(0.1*np.ones((n_neurons)))
    stim2 = nengo.Node(lambda t: 0.1*sin(10*pi*t))
    lif = nengo.Ensemble(10, 1)
    ens = nengo.Ensemble(10, 1, neuron_type=WilsonNeuron(),
        encoders=Uniform(-1, 1), gain=Uniform(-1, 1), bias=Uniform(0, 0))
    nengo.Connection(stim, ens.neurons, synapse=None)
    nengo.Connection(stim2, lif, synapse=None)
#     nengo.Connection(lif, ens,
#         solver=NoSolver(w, weights=True),
#         synapse=0.01)
    p = nengo.Probe(ens.neurons, 'voltage')
    
with nengo.Simulator(network) as sim:
    sim.run(0.2)
    
plt.plot(sim.trange(), sim.data[p])
plt.show()

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

<IPython.core.display.Javascript object>

WARNING    "I" is an internal variable of group "neurongroup_1", but also exists in the run namespace with the value array([ 0.03605035,  0.00611221,  0.08639941, -0.08506127,  0.02105894,
       -0.05393817,  0.08103297,  0.01107319, -0.07836681,  0.00107067]). The internal variable will be used. [brian2.groups.group.Group.resolve.resolution_conflict]


HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

<IPython.core.display.Javascript object>